# Building a Data Pipeline for NYC Yellow Trip Data

In this notebook, we will build a complete data pipeline using **dlt** to process NYC Yellow Taxi Trip data.

Our goal is:

→ Fetch real trip data from the Data Engineering Zoomcamp API  
→ Turn it into clean relational tables  
→ Load it into DuckDB  
→ Explore and analyze it  

We will use the **Data Engineering Zoomcamp API** as our data source and **DuckDB** as our database.

Along the way, you will learn:

- How to configure a paginated REST API source  
- How to handle pagination that stops on empty pages  
- How the Extract → Normalize → Load process works  
- How to inspect and explore NYC taxi trip data  

By the end, you will have a working pipeline processing real-world transportation data.

## 📦 Step 0: Install Dependencies

In [1]:
# install dependencies first
!pip -q install dlt[duckdb]


[notice] A new release of pip is available: 23.0.1 -> 26.0.1
[notice] To update, run: pip install --upgrade pip


In this notebook we will use:

- **dlt** to extract, normalize, and load data
- **DuckDB** as the destination database (runs locally)

DuckDB is great for data analysis because it requires no setup and no credentials.

## 📚 Step 1: Import Libraries

In this cell we import the libraries we will use throughout the notebook:

- **dlt** is the main library for building and running the pipeline
- **rest_api_source** helps us define an API source using a simple configuration
- **islice** (from `itertools`) is a small Python helper for previewing only a few records

In [2]:
import dlt
from itertools import islice
from dlt.sources.rest_api import rest_api_source

## 🔗 Step 2: Define the API Source (NYC Yellow Trip Data)

In **dlt**, a **source** is the part of your pipeline that knows how to fetch data from somewhere.
In this notebook, our source fetches NYC Yellow Taxi trip data from the **Data Engineering Zoomcamp API**.

We define the source using `rest_api_source`, which lets us describe an API in a simple Python dictionary.

**API Specifications:**
- **Base URL:** https://us-central1-dlthub-analytics.cloudfunctions.net/data_engineering_zoomcamp_api
- **Format:** Paginated JSON
- **Page Size:** 1,000 records per page
- **Pagination:** Stops when an empty page is returned

In [3]:
def nyc_yellow_trip_source():
    """
    Creates a dlt source that fetches NYC Yellow Taxi trip data
    from the Data Engineering Zoomcamp API.
    """
    return rest_api_source({
        "client": {
            "base_url": "https://us-central1-dlthub-analytics.cloudfunctions.net",
        },
        "resources": [
            {
                "name": "yellow_trips",
                "endpoint": {
                    "path": "data_engineering_zoomcamp_api",
                    "paginator": {
                        "type": "page_number",
                        "page_param": "page",
                        "total_path": None,  # Stop on empty page
                        "base_page": 1,
                    },
                },
            },
        ],
    })

## 🔧 Step 3: Create the dlt Pipeline

In [4]:
pipeline = dlt.pipeline(
    pipeline_name="nyc_yellow_trip_pipeline",
    destination="duckdb",
    dataset_name="nyc_taxi_data",
    progress="log"  # logs the pipeline run (Optional)
)

## 🔍 Understanding the Pipeline

At this point we have defined two key building blocks:

- **The source** describes where the data comes from and how to fetch it from the API.  
- **The pipeline** describes where the data should go (DuckDB) and keeps track of tables, schemas, and run history.  

---

Instead of running everything at once, we will now run the pipeline in three separate phases so you can clearly see what happens at each stage:

1. **Extract**: download raw data from the API  
2. **Normalize**: turn nested JSON into relational tables  
3. **Load**: write those tables into DuckDB  

Once these steps make sense, we will run the full workflow again using one command:

```python
pipeline.run(source)
```

## ⬇️ Step 4: Extract

Now we run the first stage of the pipeline: **Extract**.

Extract means:

- dlt sends requests to the Data Engineering Zoomcamp API
- the raw JSON responses are downloaded
- the results are stored in dlt's local working folder

At this stage, the data is **not** in DuckDB yet. We are just confirming that we successfully pulled data from the API.

In [5]:
extract_info = pipeline.extract(nyc_yellow_trip_source())

------------------------------- Extract rest_api -------------------------------
Resources: 0/1 (0.0%) | Time: 0.00s | Rate: 0.00/s
Memory usage: 178.50 MB (21.80%) | CPU usage: 0.00%

------------------------------- Extract rest_api -------------------------------
Resources: 0/1 (0.0%) | Time: 2.41s | Rate: 0.00/s
yellow_trips: 1000  | Time: 0.00s | Rate: 85598040.82/s
Memory usage: 180.50 MB (22.00%) | CPU usage: 0.00%

------------------------------- Extract rest_api -------------------------------
Resources: 0/1 (0.0%) | Time: 4.82s | Rate: 0.00/s
yellow_trips: 2000  | Time: 2.40s | Rate: 832.27/s
Memory usage: 182.13 MB (23.10%) | CPU usage: 0.00%

------------------------------- Extract rest_api -------------------------------
Resources: 0/1 (0.0%) | Time: 7.17s | Rate: 0.00/s
yellow_trips: 3000  | Time: 4.75s | Rate: 631.28/s
Memory usage: 183.38 MB (22.50%) | CPU usage: 0.00%

------------------------------- Extract rest_api -------------------------------
Resources: 0/1 (0.0%)

---

### What we will print

After extraction, we will print a small summary showing:

- which **resources** were extracted
- which **tables** will be created later
- how many rows were extracted per resource

This helps confirm that the pipeline is working before we move on to normalization.

In [6]:
load_id = extract_info.loads_ids[-1]
m = extract_info.metrics[load_id][0]

print("Resources:", list(m["resource_metrics"].keys()))
print("Tables:", list(m["table_metrics"].keys()))
print("Load ID:", load_id)
print()

for resource, rm in m["resource_metrics"].items():
    print(f"Resource: {resource}")
    print(f"rows extracted: {rm.items_count}")
    print()

Resources: ['yellow_trips']
Tables: ['yellow_trips']
Load ID: 1772402675.808388

Resource: yellow_trips
rows extracted: 10000



### What you should see after Extract

After extraction, you should see:

- **Resources:** `['yellow_trips']`  
- **Tables:** `['yellow_trips']`

The number of rows extracted depends on how many taxi trip records are available from the API. Each page contains up to 1,000 records, and the pagination continues until an empty page is returned.

---

## 🔄 Step 5: Normalize

Now we run **Normalize**. This is where dlt transforms raw JSON into a clean relational structure.

During normalization, dlt does three key things:

### 1. Adds Tracking Columns to the Main Table

dlt adds special columns to every table:
- `_dlt_id`: A unique identifier for each row
- `_dlt_load_id`: Links each row to the load job that created it

### 2. Flattens Nested Data into Child Tables

If the API response contains nested structures (like arrays or objects), dlt will flatten them into separate child tables with names like:
- `yellow_trips__nested_field`

Each child table has a `_dlt_parent_id` column that references `_dlt_id` in the parent table.

### 3. Creates Metadata Tables

dlt also creates internal tables to track pipeline state:
- `_dlt_loads`: Tracks load history (when data was loaded, status)
- `_dlt_pipeline_state`: Stores pipeline state for incremental loading
- `_dlt_version`: Tracks schema versions

In the next cell, we will print a summary showing which tables were created.

In [7]:
normalize_info = pipeline.normalize()

------------------- Normalize rest_api in 1772402675.808388 --------------------
Files: 0/2 (0.0%) | Time: 0.00s | Rate: 0.00/s
Memory usage: 188.68 MB (22.50%) | CPU usage: 0.00%

------------------- Normalize rest_api in 1772402675.808388 --------------------
Files: 0/2 (0.0%) | Time: 0.00s | Rate: 0.00/s
Items: 0  | Time: 0.00s | Rate: 0.00/s
Memory usage: 188.68 MB (22.50%) | CPU usage: 0.00%



2026-03-01 22:05:00,851|[WARNING]|4412|123160277530432|dlt|validate.py|verify_normalized_table:91|In schema `rest_api`: The following columns in table 'yellow_trips' did not receive any data during this load and therefore could not have their types inferred:
  - rate_code
  - mta_tax

Unless type hints are provided, these columns will not be materialized in the destination.
One way to provide type hints is to use the 'columns' argument in the '@dlt.resource' decorator.  For example:

@dlt.resource(columns={'rate_code': {'data_type': 'text'}})



------------------- Normalize rest_api in 1772402675.808388 --------------------
Files: 2/2 (100.0%) | Time: 1.63s | Rate: 1.23/s
Items: 0  | Time: 1.63s | Rate: 0.00/s
Memory usage: 200.46 MB (22.60%) | CPU usage: 0.00%

------------------- Normalize rest_api in 1772402675.808388 --------------------
Files: 2/2 (100.0%) | Time: 1.64s | Rate: 1.22/s
Items: 10001  | Time: 1.64s | Rate: 6112.90/s
Memory usage: 200.46 MB (22.60%) | CPU usage: 0.00%



In [8]:
load_id = normalize_info.loads_ids[-1]
m = normalize_info.metrics[load_id][0]

print("Load ID:", load_id)
print()

print("Tables created/updated:")
for table_name, tm in m["table_metrics"].items():
    # skip dlt internal tables to keep it beginner-friendly
    if table_name.startswith("_dlt"):
        continue
    print(f"  - {table_name}: {tm.items_count} rows")

Load ID: 1772402675.808388

Tables created/updated:
  - yellow_trips: 10000 rows


### What happened during Normalize?

After running `pipeline.normalize()`, the data has been transformed from raw JSON into a **relational structure**.

If the NYC Yellow Trip data contains nested fields, you may see additional child tables created automatically by dlt.

---

### Schema Visualization

dlt can render the schema as a visual diagram. Run the next cell to see the table relationships:

In [9]:
# Display schema
pipeline.default_schema

<dlt.Schema(name='rest_api', version=2, tables=['_dlt_version', '_dlt_loads', 'yellow_trips', '_dlt_pipeline_state'], version_hash='n/SI1iwtPunmyqcE0CIa+j7Lr5bzNoTmITZkuHjVQmw=')>

## 📤 Step 6: Load

Now we run the final stage of the pipeline: **Load**.

Load means:

- dlt creates tables in DuckDB (if they do not already exist)
- the normalized rows are inserted into those tables
- the pipeline records the load in its internal tracking tables

In [10]:
load_info = pipeline.load()

---------------------- Load rest_api in 1772402675.808388 ----------------------
Jobs: 0/2 (0.0%) | Time: 0.00s | Rate: 0.00/s
Memory usage: 215.55 MB (22.50%) | CPU usage: 0.00%

---------------------- Load rest_api in 1772402675.808388 ----------------------
Jobs: 1/2 (50.0%) | Time: 2.86s | Rate: 0.35/s
Memory usage: 355.76 MB (23.20%) | CPU usage: 0.00%

---------------------- Load rest_api in 1772402675.808388 ----------------------
Jobs: 2/2 (100.0%) | Time: 3.00s | Rate: 0.67/s
Memory usage: 231.32 MB (22.60%) | CPU usage: 0.00%



After this step, the data is fully stored in the database and ready to query.

At this point:

- The `yellow_trips` table contains NYC Yellow Taxi trip records
- Any related child tables contain exploded nested data
- Everything is now queryable using `pipeline.dataset()` or SQL

This is the moment where the data officially moves from "pipeline processing" into a database you can explore.

## 🚀 Step 7: Run the Full Pipeline

Now that we have walked through each step individually, we can run the entire workflow using a single command:

In [11]:
load_info = pipeline.run(nyc_yellow_trip_source())

------------------------------- Extract rest_api -------------------------------
Resources: 0/1 (0.0%) | Time: 0.00s | Rate: 0.00/s
Memory usage: 231.53 MB (22.40%) | CPU usage: 0.00%

------------------------------- Extract rest_api -------------------------------
Resources: 0/1 (0.0%) | Time: 2.82s | Rate: 0.00/s
yellow_trips: 1000  | Time: 0.00s | Rate: 161319384.62/s
Memory usage: 231.78 MB (22.40%) | CPU usage: 0.00%

------------------------------- Extract rest_api -------------------------------
Resources: 0/1 (0.0%) | Time: 5.15s | Rate: 0.00/s
yellow_trips: 2000  | Time: 2.33s | Rate: 859.16/s
Memory usage: 233.28 MB (22.40%) | CPU usage: 0.00%

------------------------------- Extract rest_api -------------------------------
Resources: 0/1 (0.0%) | Time: 7.52s | Rate: 0.00/s
yellow_trips: 3000  | Time: 4.70s | Rate: 638.21/s
Memory usage: 234.53 MB (22.40%) | CPU usage: 0.00%

------------------------------- Extract rest_api -------------------------------
Resources: 0/1 (0.0%

2026-03-01 22:05:31,276|[WARNING]|4412|123160277530432|dlt|validate.py|verify_normalized_table:91|In schema `rest_api`: The following columns in table 'yellow_trips' did not receive any data during this load and therefore could not have their types inferred:
  - rate_code
  - mta_tax

Unless type hints are provided, these columns will not be materialized in the destination.
One way to provide type hints is to use the 'columns' argument in the '@dlt.resource' decorator.  For example:

@dlt.resource(columns={'rate_code': {'data_type': 'text'}})



------------------- Normalize rest_api in 1772402704.427597 --------------------
Files: 1/1 (100.0%) | Time: 1.49s | Rate: 0.67/s
Items: 0  | Time: 1.49s | Rate: 0.00/s
Memory usage: 251.03 MB (22.60%) | CPU usage: 0.00%

------------------- Normalize rest_api in 1772402704.427597 --------------------
Files: 1/1 (100.0%) | Time: 1.50s | Rate: 0.67/s
Items: 10000  | Time: 1.50s | Rate: 6676.47/s
Memory usage: 251.03 MB (22.60%) | CPU usage: 0.00%

---------------------- Load rest_api in 1772402704.427597 ----------------------
Jobs: 0/1 (0.0%) | Time: 0.00s | Rate: 0.00/s
Memory usage: 251.03 MB (22.60%) | CPU usage: 0.00%

---------------------- Load rest_api in 1772402704.427597 ----------------------
Jobs: 0/1 (0.0%) | Time: 2.66s | Rate: 0.00/s
Memory usage: 373.64 MB (23.20%) | CPU usage: 0.00%

---------------------- Load rest_api in 1772402704.427597 ----------------------
Jobs: 1/1 (100.0%) | Time: 2.85s | Rate: 0.35/s
Memory usage: 243.95 MB (22.60%) | CPU usage: 0.00%



### What does `pipeline.run()` do?

`pipeline.run()` simply combines the three steps we already executed manually:

1. **Extract** – fetch data from the NYC Yellow Trip API
2. **Normalize** – convert nested JSON into relational tables
3. **Load** – write those tables into DuckDB

In other words, this:

```python
pipeline.run(source)
```

is equivalent to:

```python
pipeline.extract(source)
pipeline.normalize()
pipeline.load()
```

There is no hidden magic. It just runs the full ELT process in order.

## 🔎 Step 8: Inspect the Loaded Data

Now that the data is loaded into DuckDB, we can inspect it using `pipeline.dataset()`.

This gives us a convenient Python interface for exploring the tables that dlt created, without writing SQL.

---

### List available tables

First, let's see what tables exist in the dataset:

In [12]:
ds = pipeline.dataset()

In [13]:
ds.tables

['yellow_trips', '_dlt_version', '_dlt_loads', '_dlt_pipeline_state']

### Preview the yellow_trips table

Let's look at the first few rows of the main table:

In [14]:
df = ds.yellow_trips.df()      # main table
df.head(10)

,end_lat,end_lon,fare_amt,passenger_count,payment_type,start_lat,start_lon,tip_amt,tolls_amt,total_amt,trip_distance,trip_dropoff_date_time,trip_pickup_date_time,surcharge,vendor_name,_dlt_load_id,_dlt_id,store_and_forward
0,40.742963,-73.980072,45.0,1,Credit,40.641525,-73.787442,9.0,4.15,58.15,17.52,2009-06-14 23:48:00+00:00,2009-06-14 23:23:00+00:00,0.0,VTS,1772402675.808388,/hWsRZoUjwv1bg,NaN
1,40.740187,-74.005698,6.5,1,Credit,40.722065,-74.009767,1.0,0.00,8.50,1.56,2009-06-18 17:43:00+00:00,2009-06-18 17:35:00+00:00,1.0,VTS,1772402675.808388,MAIMtuN2gazswQ,NaN
2,40.718043,-74.004745,12.5,5,Credit,40.761945,-73.983038,2.0,0.00,15.50,3.37,2009-06-10 18:27:00+00:00,2009-06-10 18:08:00+00:00,1.0,VTS,1772402675.808388,xbDaR/VLNIU6WQ,NaN
3,40.739637,-73.985233,4.9,1,CASH,40.749802,-73.992247,0.0,0.00,5.40,1.11,2009-06-14 23:58:00+00:00,2009-06-14 23:54:00+00:00,0.5,VTS,1772402675.808388,eOLOvg2rP7P5wA,NaN
4,40.730032,-73.852693,25.7,1,CASH,40.776825,-73.949233,0.0,4.15,29.85,11.09,2009-06-13 13:23:00+00:00,2009-06-13 13:01:00+00:00,0.0,VTS,1772402675.808388,gLsiw//6xaHxhQ,NaN
5,40.777537,-73.976860,7.3,2,Credit,40.790582,-73.953652,2.0,0.00,10.30,2.10,2009-06-10 19:52:00+00:00,2009-06-10 19:43:00+00:00,1.0,VTS,1772402675.808388,+L7lncjVJ9HdiQ,NaN
6,40.770277,-73.962125,3.7,1,Credit,40.767147,-73.966408,1.0,0.00,5.20,0.40,2009-06-10 20:09:00+00:00,2009-06-10 20:06:00+00:00,0.5,VTS,1772402675.808388,V3rKcpuhs+d7ow,NaN
7,40.774043,-73.951465,8.1,2,CASH,40.761750,-73.977773,0.0,0.00,8.60,2.24,2009-06-14 21:08:00+00:00,2009-06-14 20:57:00+00:00,0.5,VTS,1772402675.808388,LEtBQyVV8s5rtg,NaN
8,40.777985,-73.943683,6.1,1,CASH,40.766355,-73.959832,0.0,0.00,6.10,1.48,2009-06-14 12:56:00+00:00,2009-06-14 12:49:00+00:00,0.0,VTS,1772402675.808388,grrCGxjTS6DpiQ,NaN
9,40.720052,-74.009823,8.9,1,CASH,40.751327,-73.987588,0.0,0.00,9.90,2.72,2009-06-10 18:13:00+00:00,2009-06-10 18:03:00+00:00,1.0,VTS,1772402675.808388,h0m9FAocyYSBHw,NaN


### Basic Data Analysis

Let's explore some basic statistics about the NYC Yellow Taxi trips:

In [15]:
# Total number of trips
print(f"Total trips: {len(df)}")
print()

# Display data types 
print("Data types:")
print(df.dtypes)

Total trips: 20000

Data types:
end_lat                               float64
end_lon                               float64
fare_amt                              float64
passenger_count                         int64
payment_type                           object
start_lat                             float64
start_lon                             float64
tip_amt                               float64
tolls_amt                             float64
total_amt                             float64
trip_distance                         float64
trip_dropoff_date_time    datetime64[us, UTC]
trip_pickup_date_time     datetime64[us, UTC]
surcharge                             float64
vendor_name                            object
_dlt_load_id                           object
_dlt_id                                object
store_and_forward                     float64
dtype: object


## 💡 Conclusion

### What we accomplished

In this notebook, we built a complete data pipeline that:

✔ Fetches NYC Yellow Taxi trip data from a REST API  
✔ Handles pagination automatically (stops on empty page)  
✔ Normalizes JSON into relational tables  
✔ Loads data into DuckDB  
✔ Provides easy data inspection and analysis  

---

### What dlt handled for us

✔ API requests with proper pagination  
✔ JSON to relational normalization  
✔ Table creation with proper schemas  
✔ Database loading  
✔ Simple dataset inspection  

---

### Key Takeaways

• **Extract** downloads raw data from the API  
• **Normalize** converts JSON into clean relational tables  
• **Load** writes data into the destination database  
• `pipeline.run()` executes all three steps in sequence  
• The resulting data is easily queryable via `pipeline.dataset()` or SQL  



## Question 1: What is the start date and end date of the dataset?

In [19]:
df['trip_pickup_date_time'].min(), df['trip_dropoff_date_time'].max()

(Timestamp('2009-06-01 11:33:00+0000', tz='UTC'),
 Timestamp('2009-07-01 00:03:00+0000', tz='UTC'))

## Question 2. What proportion of trips are paid with credit card? 



In [22]:
df[df['payment_type'] == 'Credit'].shape[0] / len(df)

0.2666

## Question 3. What is the total amount of money generated in tips? 

In [44]:
total_tips = (df['tip_amt']).sum()
print(f"Total amount of money generated in tips: ${total_tips:,.2f}")

Total amount of money generated in tips: $12,126.82
